In [1]:
library(devtools)
library("fgsea")
library("here")
library(devtools)
library(tidyverse)
library(Matrix)
library(magrittr) # needs to be run every time you start R and want to use %>%
library(dplyr)  

Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis

here() starts at /home/jpm73279/Mendieta_et_al_comparative_single_cell

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.0     
✔ ggplot2   3.4.3          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘

# Running On Zea Mays

In [2]:
convertTerms <- function(x){
    out <- lapply(seq(1:nrow(x)), function(z){
        gids <- strsplit(as.character(x$V2[z]), "\\,")
        gids <- do.call(c, gids)
        return(gids)
    })
    names(out) <- as.character(x$V1)
    return(out)
}

zm_func_genes <- read.table("/home/jpm73279/genome_downloads/Zm-B73-REFERENCE-NAM-5.0/biological_process.gmt.txt")
zm_functional_annotation <- convertTerms(zm_func_genes)



In [20]:
## BS deseq2 results
zm_bs_denovo <- read.table("/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers/Zm.de_novo_genes/Zm.V3.final_annotation.bundle_sheath_ncell_2870_deseq_2_results.tsv",
                          header = TRUE)
    

In [27]:
read_table_run_FGSEA <- function(de_seq2_input, location, functional_annotations){
    
    input_file_name <- here(location, de_seq2_input)
    zm_bs_denovo <- read.table(input_file_name,
                          header = TRUE)
    
    zm_bs_denovo.selected <- zm_bs_denovo  %>% 
        dplyr::select(gene_name, stat)

    ranks <- deframe(zm_bs_denovo.selected)
    
    fgseaRes <- fgsea(pathways=zm_functional_annotation, stats=ranks, 
                  nperm=10000,
                 minSize = 10, maxSize = 600)
    
    fgseaResTidy <- fgseaRes %>%
        as_tibble() %>%
        arrange(NES)
        
    
    fgseaResTidy$celltype <- de_seq2_input

    return(fgseaResTidy)
}

In [28]:
working_location <- "/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers/Zm.de_novo_genes"
deseq2_files <- c("Zm.V4.final_annotation.bundle_sheath_ncell_3088.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.companion_cells_ncell_732.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.developing_mesophyll_ncell_516.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.epidermis_ncell_3607.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.ground_meristem_ncell_1329.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.mesophyll_ncell_4766.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.procambium_ncell_367.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.protoderm_ncell_2107.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.sieve_elements_ncell_1520.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.stomatal_precursor_ncell_217.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.subsidiary_cells_ncell_515.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.subsidiary_mother_cells_ncell_326.upregulated_genes.deseq2_output.tsv",
"Zm.V4.final_annotation.unknown_ncell_128.upregulated_genes.deseq2_output.tsv")

all_gathered_cell_type_FGSEA <- lapply(deseq2_files, read_table_run_FGSEA, location = working_location, functional_annotations = zm_functional_annotation)

Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.37% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”
Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in 

In [ ]:
read_table_run_FGSEA()

In [29]:
zm_cell_type_FGSEA <- bind_rows(all_gathered_cell_type_FGSEA)

In [30]:
head(zm_functional_annotation)

$mitochondrial_genome_maintenance
[1] "Zm00001eb085130" "Zm00001eb339190" "Zm00001eb414740" "Zm00001eb421290"
[5] "Zm00001eb426210" "Zm00001eb431790" "Zm00001eb294240" "Zm00001eb141480"

$reproduction
   [1] "Zm00001eb065570" "Zm00001eb065660" "Zm00001eb065740" "Zm00001eb065760"
   [5] "Zm00001eb065940" "Zm00001eb065970" "Zm00001eb066340" "Zm00001eb066640"
   [9] "Zm00001eb066780" "Zm00001eb066790" "Zm00001eb066830" "Zm00001eb067080"
  [13] "Zm00001eb067210" "Zm00001eb067270" "Zm00001eb067440" "Zm00001eb067510"
  [17] "Zm00001eb067800" "Zm00001eb068020" "Zm00001eb068390" "Zm00001eb068440"
  [21] "Zm00001eb068550" "Zm00001eb068720" "Zm00001eb068710" "Zm00001eb068860"
  [25] "Zm00001eb068880" "Zm00001eb068930" "Zm00001eb069270" "Zm00001eb069290"
  [29] "Zm00001eb069410" "Zm00001eb069440" "Zm00001eb069480" "Zm00001eb069490"
  [33] "Zm00001eb069500" "Zm00001eb069530" "Zm00001eb069550" "Zm00001eb069670"
  [37] "Zm00001eb069690" "Zm00001eb070050" "Zm00001eb070270" "Zm00001eb070280"
  [41] "Zm00001eb070450" "Zm00001eb070760" "Zm00001eb070860" "Zm00001eb070960"
  [45] "Zm00001eb070980" "Zm00001eb071030" "Zm00001eb071550" "Zm00001eb071970"
  [49] "Zm00001eb071980" "Zm00001eb071990" "Zm00001eb072270" "Zm00001eb072410"
  [53] "Zm00001eb072420" "Zm00001eb072800" "Zm00001eb072940" "Zm00001eb073010"
  [57] "Zm00001eb073110" "Zm00001eb073240" "Zm00001eb073290" "Zm00001eb073370"
  [61] "Zm00001eb073540" "Zm00001eb073810" "Zm00001eb074060" "Zm00001eb074140"
  [65] "Zm00001eb074180" "Zm00001eb074570" "Zm00001eb074930" "Zm00001eb074940"
  [69] "Zm00001eb074990" "Zm00001eb075050" "Zm00001eb075140" "Zm00001eb075200"
  [73] "Zm00001eb075250" "Zm00001eb075420" "Zm00001eb075520" "Zm00001eb075640"
  [77] "Zm00001eb075800" "Zm00001eb075810" "Zm00001eb076080" "Zm00001eb076170"
  [81] "Zm00001eb076180" "Zm00001eb076210" "Zm00001eb076420" "Zm00001eb076460"
  [85] "Zm00001eb076670" "Zm00001eb076890" "Zm00001eb077250" "Zm00001eb077610"
  [89] "Zm00001eb077770" "Zm00001eb077840" "Zm00001eb077960" "Zm00001eb077980"
  [93] "Zm00001eb078280" "Zm00001eb078350" "Zm00001eb078540" "Zm00001eb078720"
  [97] "Zm00001eb078730" "Zm00001eb078790" "Zm00001eb078880" "Zm00001eb078900"
 [101] "Zm00001eb078980" "Zm00001eb079000" "Zm00001eb079310" "Zm00001eb079490"
 [105] "Zm00001eb079500" "Zm00001eb079670" "Zm00001eb079950" "Zm00001eb080690"
 [109] "Zm00001eb081350" "Zm00001eb081420" "Zm00001eb081530" "Zm00001eb081610"
 [113] "Zm00001eb081630" "Zm00001eb081720" "Zm00001eb082060" "Zm00001eb082160"
 [117] "Zm00001eb082270" "Zm00001eb082590" "Zm00001eb082700" "Zm00001eb082750"
 [121] "Zm00001eb082810" "Zm00001eb082920" "Zm00001eb083200" "Zm00001eb083290"
 [125] "Zm00001eb083630" "Zm00001eb083900" "Zm00001eb084030" "Zm00001eb084190"
 [129] "Zm00001eb084710" "Zm00001eb084990" "Zm00001eb085030" "Zm00001eb085190"
 [133] "Zm00001eb085270" "Zm00001eb085290" "Zm00001eb085530" "Zm00001eb086560"
 [137] "Zm00001eb086710" "Zm00001eb087050" "Zm00001eb087250" "Zm00001eb087500"
 [141] "Zm00001eb088350" "Zm00001eb088500" "Zm00001eb088510" "Zm00001eb088790"
 [145] "Zm00001eb088950" "Zm00001eb089080" "Zm00001eb089580" "Zm00001eb089960"
 [149] "Zm00001eb090320" "Zm00001eb090350" "Zm00001eb090620" "Zm00001eb090890"
 [153] "Zm00001eb091010" "Zm00001eb091080" "Zm00001eb091230" "Zm00001eb091430"
 [157] "Zm00001eb091890" "Zm00001eb091940" "Zm00001eb093110" "Zm00001eb093140"
 [161] "Zm00001eb093200" "Zm00001eb093280" "Zm00001eb093290" "Zm00001eb093300"
 [165] "Zm00001eb093320" "Zm00001eb093380" "Zm00001eb093450" "Zm00001eb093710"
 [169] "Zm00001eb093890" "Zm00001eb093980" "Zm00001eb094220" "Zm00001eb094310"
 [173] "Zm00001eb094350" "Zm00001eb094400" "Zm00001eb094420" "Zm00001eb094550"
 [177] "Zm00001eb094560" "Zm00001eb094840" "Zm00001eb094860" "Zm00001eb094890"
 [181] "Zm00001eb094910" "Zm00001eb095100" "Zm00001eb095120" "Zm00001eb095240"
 [185] "Zm00001eb095250" "Zm00001eb095370" "Zm00001eb095560" "Zm00001eb095660"
 [189] "Zm00001eb095750" "Zm00001eb095960" "Zm00001eb096590" "Zm00001eb096820"
 [193] 

In [31]:
fitlered_save <- zm_cell_type_FGSEA  %>% 
    group_by(celltype)  %>% 
    mutate_at(c('pval'), as.numeric)  %>% 
    dplyr::filter(NES > 0 & pval < .05) %>% 
    dplyr::arrange(pval, descending = TRUE)  %>% 
    dplyr::arrange(celltype)  %>% 
    mutate(celltype = str_replace(celltype, "Zm.V4.final_annotation.", ""))  %>% 
    mutate(celltype = str_replace(celltype, ".upregulated_genes.deseq2_output.tsv", ""))  %>% 
    dplyr::select(-leadingEdge)

write_delim(fitlered_save, "/home/jpm73279/Mendieta_et_al_comparative_single_cell/metrics/supplament/Zm.FGSEA_results.tsv", delim = "\t")

In [91]:
head(fitlered_save)

pathway,pval,padj,ES,NES,nMoreExtreme,size,celltype
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
photosynthesis,9.999e-05,0.004638827,0.4911888,1.982166,0,146,bundle_sheath_ncell_2870
ion_transmembrane_transport,9.999e-05,0.004638827,0.4423916,1.798202,0,164,bundle_sheath_ncell_2870
generation_of_precursor_metabolites_and_energy,9.999e-05,0.004638827,0.4256397,1.754672,0,213,bundle_sheath_ncell_2870
anion_transport,9.999e-05,0.004638827,0.4208470,1.710618,0,165,bundle_sheath_ncell_2870
transmembrane_transport,9.999e-05,0.004638827,0.3851061,1.608706,0,283,bundle_sheath_ncell_2870
ion_transport,9.999e-05,0.004638827,0.3776028,1.590161,0,348,bundle_sheath_ncell_2870
